Here do also the t-test for the results of the different models by metric.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from utils import display_results, display_structure


In [ ]:
# import any json file from the results folders

file = "./results/targCls_3_music_openl3_music_voice_False_TotCls_5_dropNs_True_rep_5_fold_5.json"

with open(file) as f:
    data = json.load(f)
    print(file)
    display_results(data)
    #print(data["config"]["classifications"].keys())


In [ ]:
rep = 5
fold = 5
lvl_0 = ["Baseline", "FiLM-ed"]
lvl_1 = ["Binary", "Ternary"]
lvl_2 = ["Voice", "No voice"]
numerical_values = ["Target F1", "Avg. secondary F1", "Avg. R2 emotions", "Avg. r emotions", "Avg. R2 mid-level", "Avg. r mid-level"]

columns = ["Embeddings"] + numerical_values
dict_of_dfs = {k0: {k1: {k2: pd.DataFrame(columns=columns) for k2 in lvl_2} for k1 in lvl_1} for k0 in lvl_0}

for suffix, l0 in zip(["", "_filmed"], lvl_0):
    temp_df = dict_of_dfs[l0]
    for fn in glob.glob(f"./results{suffix}/*_rep_{rep}_fold_{fold}.json"):
        with open(fn) as f:
            results = json.load(f)

        # aggregate secondary f1 scores
        f1_secondary_means = [v['mean'] for k,v in results["f1"].items() if k != 'target']
        f1_secondary_stds = [v['std'] for k,v in results["f1"].items() if k != 'target']
        results["average"]["secondary f1"] = { "mean": np.mean(f1_secondary_means), "std": np.mean(f1_secondary_stds)}

        first_level = "Binary" if len(results["config"]["classifications"]["target"]) == 2 else "Ternary"
        second_level = "Voice" if results["config"]["voice"] else "No voice"

        row = {
            "Embeddings": results["config"]["which_embeddings"],
            "Target F1": f'{results["f1"]["target"]["mean"]:.2f} ± {results["f1"]["target"]["std"]:.2f}',
            "Avg. secondary F1": f'{results["average"]["secondary f1"]["mean"]:.2f} ± {results["average"]["secondary f1"]["std"]:.2f}',
            "Avg. R2 emotions": f'{results["average"]["emo_r2"]["mean"]:.2f} ± {results["average"]["emo_r2"]["std"]:.2f}',
            "Avg. r emotions": f'{results["average"]["emo_pears"]["mean"]:.2f} ± {results["average"]["emo_pears"]["std"]:.2f}',
            "Avg. R2 mid-level": f'{results["average"]["mid_r2"]["mean"]:.2f} ± {results["average"]["mid_r2"]["std"]:.2f}',
            "Avg. r mid-level": f'{results["average"]["mid_pears"]["mean"]:.2f} ± {results["average"]["mid_pears"]["std"]:.2f}'
        }

        df = temp_df[first_level][second_level].copy()
        df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)
        temp_df[first_level][second_level] = df

for l0 in lvl_0:
    for l1 in lvl_1:
        for l2 in lvl_2:
            dict_of_dfs[l0][l1][l2] = dict_of_dfs[l0][l1][l2].sort_values(by="Embeddings").set_index("Embeddings")

In [ ]:
for l1 in lvl_1:
    print(f"{l1}")
    for l2 in lvl_2:
        print(f"{l2}")
        df = dict_of_dfs["Baseline"][l1][l2].copy()
        display(df.round(2))
        #display(df_std)
        print("")
    print("_____________________________________________________________")

    lvl_0 = ["Baseline", "FiLM-ed"]
    lvl_1 = ["Binary", "Ternary"]
    lvl_2 = ["Voice", "No voice"]
    numerical_values = ["Target F1", "Avg. secondary F1", "Avg. R2 emotions", "Avg. r emotions", "Avg. R2 mid-level", "Avg. r mid-level"]

    columns = ["Embeddings"] + [f"{col} mean" for col in numerical_values] + [f"{col} std" for col in numerical_values]
    dict_of_dfs = {k0: {k1: {k2: pd.DataFrame(columns=columns) for k2 in lvl_2} for k1 in lvl_1} for k0 in lvl_0}

    for suffix, l0 in zip(["", "_filmed"], lvl_0):
        temp_dict = dict_of_dfs[l0]
        for fn in glob.glob(f"./results{suffix}/*.json"):
            with open(fn) as f:
                results = json.load(f)

            # aggregate secondary f1 scores
            f1_secondary_means = [v['mean'] for k,v in results["f1"].items() if k != 'target']
            f1_secondary_stds = [v['std'] for k,v in results["f1"].items() if k != 'target']
            results["average"]["secondary f1"] = { "mean": np.mean(f1_secondary_means), "std": np.mean(f1_secondary_stds)}

            first_level = "Binary" if len(results["config"]["classifications"]["target"]) == 2 else "Ternary"
            second_level = "Voice" if results["config"]["voice"] else "No voice"

            row = {
                "Embeddings": results["config"]["which_embeddings"],
                "Target F1 mean": results["f1"]["target"]["mean"],
                "Target F1 std": results["f1"]["target"]["std"],
                "Avg. secondary F1 mean": results["average"]["secondary f1"]["mean"],
                "Avg. secondary F1 std": results["average"]["secondary f1"]["std"],
                "Avg. R2 emotions mean": results["average"]["emo_r2"]["mean"],
                "Avg. R2 emotions std": results["average"]["emo_r2"]["std"],
                "Avg. r emotions mean": results["average"]["emo_pears"]["mean"],
                "Avg. r emotions std": results["average"]["emo_pears"]["std"],
                "Avg. R2 mid-level mean": results["average"]["mid_r2"]["mean"],
                "Avg. R2 mid-level std": results["average"]["mid_r2"]["std"],
                "Avg. r mid-level mean": results["average"]["mid_pears"]["mean"],
                "Avg. r mid-level std": results["average"]["mid_pears"]["std"]
            }

            df = temp_dict[first_level][second_level].copy()
            df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)
            
            temp_dict[first_level][second_level] = df
        
        dict_of_dfs[l0] = temp_dict

    for l0 in lvl_0:
        for l1 in lvl_1:
            for l2 in lvl_2:
                dict_of_dfs[l0][l1][l2] = dict_of_dfs[l0][l1][l2].sort_values(by="Embeddings").set_index("Embeddings")
